<a href="https://colab.research.google.com/github/kimdonggyu2008/2024_2_Capstone/blob/main/translator_train_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#번역 모델(helsinki) 학습 및 추론 코드

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from transformers import MarianMTModel, MarianTokenizer
from datasets import load_dataset, DatasetDict
import pickle
import torch
import pandas as pd
import os
from datasets import Dataset
from sklearn.model_selection import train_test_split

#데이터 병합 및 토크나이저, 처리(필요시 사용)

In [ ]:

# 3. 데이터셋 로드 (예시 데이터셋)
df = pd.read_csv('/content/drive/MyDrive/translator/data/train.csv')#train.csv파일 있으면 루트 변경
dataset = Dataset.from_pandas(df)


In [ ]:
# # combined_df를 Dataset으로 변환
# dataset = Dataset.from_pandas(df)

# # train, test 데이터셋 분리 (예시로 10%를 테스트 세트로 분리)
# train_test = dataset.train_test_split(test_size=0.1)
# datasets = DatasetDict({
#     'train': train_test['train'],
#     'test': train_test['test']
# })

# # 4. 전처리 함수 (한국어 -> 영어 번역)
# def preprocess_function(examples):
#     inputs = examples['ko']  # 'ko' 열에서 한국어 텍스트를 가져옴
#     targets = examples['en']  # 'en' 열에서 영어 번역 텍스트를 가져옴
#     model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

#     # 타겟 텍스트도 토크나이징
#     labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

# # # 데이터셋을 전처리 및 토크나이징 (datasets가 이미 로드된 상태에서)
# tokenized_datasets = datasets.map(preprocess_function, batched=True)


In [ ]:
#토크나이징된 데이터셋을 4개로 나눔


# # 5. 4개의 파트로 분할하기 위해 총 데이터셋 크기 계산
# dataset_length = len(tokenized_datasets['train'])

# #파일을 4개로 나누어서 저장, 나중에 각각 불러와서 에폭마다 돌려야할듯
# split_size = dataset_length // 4

# # 7. 각 파트를 pkl 파일로 저장
# for i in range(4):
#     start_idx = i * split_size
#     end_idx = (i + 1) * split_size if i < 3 else dataset_length  # 마지막 파트는 끝까지

#     part_dataset = tokenized_datasets['train'].select(range(start_idx, end_idx))

#     output_file = f"/content/drive/MyDrive/combined/캡스톤 데이터 전처리 파일/translator/processed_dataset_part_{i+1}.pkl"
#     with open(output_file, 'wb') as f:
#         pickle.dump(part_dataset, f)

#     print(f"Tokenized dataset part {i+1} saved to {output_file}")

#학습 세팅(토크나이징 된 데이터 받음)


In [ ]:
# #모델과 데이터셋 설정, 처음 학습시킬때, 온라인 모델 받아와서 사용
# tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-tc-big-en-ko")
# model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-tc-big-en-ko")

In [ ]:
#모델과 데이터셋 설정, 로컬로 불러와서 학습시키는 경우
model_name="/content/drive/MyDrive/translator/model_part_1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
# 4. 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/translator/checkpoints",  # 모델 및 체크포인트가 저장될 디렉토리
    evaluation_strategy="epoch",  # 에폭마다 평가
    save_strategy="epoch",  # 에폭마다 체크포인트 저장
    save_total_limit=3,  # 최근 3개의 체크포인트만 유지
    logging_steps=100,  # 로그를 출력할 스텝 간격
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,  # 각 파일별로 5 에폭씩 학습
    weight_decay=0.01,
    load_best_model_at_end=True,  # 학습 종료 시 가장 성능이 좋은 모델 불러오기
    save_on_each_node=False,  # 모든 노드에서 체크포인트를 저장하지 않음 (분산 학습 시)
    logging_dir="/content/drive/MyDrive/translator/logging",  # 로그를 저장할 디렉토리
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
model_checkpoint_path = "/content/drive/MyDrive/translator/checkpoints"

In [ ]:
# #모델이 존재하는 경우 사용할 필요가 없음
#last_checkpoint = None
#if os.path.isdir(model_checkpoint_path) and os.listdir(model_checkpoint_path):
#    last_checkpoint = Trainer.get_last_checkpoint(model_checkpoint_path)
#    if last_checkpoint is not None:
#        print(f"체크포인트에서 이어서 학습을 시작합니다: {last_checkpoint}")
#    else:
#        print("체크포인트가 없습니다. 새로 학습을 시작합니다.")
#else:
 #   print("체크포인트 디렉토리가 존재하지 않습니다. 새로 학습을 시작합니다.")


#학습(학습 파일은 여러개로 되어있음)

In [ ]:
# # 3. 데이터셋 리스트
# file_paths = [
#     "/content/drive/MyDrive/combined/캡스톤 데이터 전처리 파일/translator/processed_dataset_part_1.pkl",
#     "/content/drive/MyDrive/combined/캡스톤 데이터 전처리 파일/translator/processed_dataset_part_2.pkl",
#     "/content/drive/MyDrive/combined/캡스톤 데이터 전처리 파일/translator/processed_dataset_part_3.pkl",
#     "/content/drive/MyDrive/combined/캡스톤 데이터 전처리 파일/translator/processed_dataset_part_4.pkl"
# ]

In [ ]:
# def load_preprocessed_data(file_path):
#     with open(file_path, 'rb') as f:
#         tokenized_datasets = pickle.load(f)
#     return tokenized_datasets

In [ ]:
# def load_preprocessed_data(file_path, split_ratio=0.8):
#     with open(file_path, 'rb') as f:
#         tokenized_datasets = pickle.load(f)

#     # 학습 데이터와 평가 데이터를 나눔 (default: 80% train, 20% eval)
#     train_size = int(split_ratio * len(tokenized_datasets))
#     train_dataset = tokenized_datasets[:train_size]
#     eval_dataset = tokenized_datasets[train_size:]

#     return train_dataset, eval_dataset


In [ ]:
import pickle
from datasets import DatasetDict

def load_preprocessed_data(file_path, split_ratio=0.8):
    with open(file_path, 'rb') as f:
        tokenized_datasets = pickle.load(f)

    # tokenized_datasets가 Dataset 형식일 경우 train_test_split 사용
    if isinstance(tokenized_datasets, DatasetDict):
        split_data = tokenized_datasets.train_test_split(test_size=(1 - split_ratio))
        train_dataset = split_data['train']
        eval_dataset = split_data['test']
    else:
        # 리스트나 배열인 경우 슬라이싱으로 나눔
        train_size = int(split_ratio * len(tokenized_datasets))
        train_dataset = tokenized_datasets[:train_size]
        eval_dataset = tokenized_datasets[train_size:]

    return train_dataset, eval_dataset


In [ ]:
def load_tokenized_data(file_path):
    with open(file_path, 'rb') as f:
        tokenized_datasets = pickle.load(f)
    return tokenized_datasets

In [ ]:
import pickle
from transformers import Trainer, TrainingArguments
import pandas as pd
from datasets import Dataset
import pickle

# 전처리 함수 (동일하게 사용)
def preprocess_function(examples):
    inputs = examples['ko']  # 'ko' 열에서 한국어 텍스트를 가져옴
    targets = examples['en']  # 'en' 열에서 영어 번역 텍스트를 가져옴
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # 타겟 텍스트도 토크나이징
    labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir=f"/content/drive/MyDrive/translator/checkpoints",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
)

# 6개의 데이터셋을 순차적으로 불러와 학습 진행
#한 데이터셋의 학습이 끝나면 하나씩 지우고 길이 줄이기 1~7 -> 2~7 -> 3~7...
for i in range(2, 7):
    print(f"Loading dataset part {i}...")

    # 저장된 파일 불러오기
    with open(f"/content/drive/MyDrive/translator/data/part_{i}.pkl", "rb") as f:
        part = pickle.load(f)

    # 파트마다 전처리 및 토큰화
    tokenized_part = part.map(preprocess_function, batched=True)

    # Trainer 설정
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_part,  # 현재 파트 데이터셋으로 학습
        eval_dataset=tokenized_part,   # 평가도 동일한 데이터셋으로 설정
        data_collator=data_collator,
        tokenizer=tokenizer
    )

    # api키는 이걸로 넣으면 됨 513a1f0c050fa7f60a76b5232e904d8df397082e
    # 학습 시작
    print(f"Training on dataset part {i}...")
    try:
        trainer.train()
    except KeyboardInterrupt:
        print(f"Training interrupted during part {i}. Last checkpoint saved.")

    # 체크포인트 저장
    trainer.save_model(f"/content/drive/MyDrive/translator/model/model_part_{i}")
    print(f"Model checkpoint saved after part {i}")


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Loading dataset part 2...


Map:   0%|          | 0/138887 [00:00<?, ? examples/s]

Training on dataset part 2...


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


In [ ]:

# # 1. CSV 파일 로드
# df = pd.read_csv('/content/drive/MyDrive/translator/data/train.csv')

# # 2. Huggingface Dataset으로 변환
# dataset = Dataset.from_pandas(df)

# # 3. 데이터를 4개로 나누기
# split_ratio = [0.25, 0.25, 0.25, 0.25]  # 데이터를 4등분할 비율
# datasets_split = dataset.train_test_split(test_size=0.75)  # 1/4 데이터 추출
# part_1 = datasets_split['train']  # 첫 번째 1/4
# remaining_data = datasets_split['test']

# # 나머지 데이터를 다시 나눔
# datasets_split = remaining_data.train_test_split(test_size=2/3)  # 3/4 남은 데이터에서 1/4 추출
# part_2 = datasets_split['train']
# remaining_data = datasets_split['test']

# datasets_split = remaining_data.train_test_split(test_size=1/2)  # 2/4 남은 데이터에서 1/4 추출
# part_3 = datasets_split['train']
# part_4 = datasets_split['test']

# # 4. 각 파트마다 5 에폭씩 학습
# parts = [part_1, part_2, part_3, part_4]

# # 전처리 함수
# def preprocess_function(examples):
#     inputs = examples['ko']  # 'ko' 열에서 한국어 텍스트를 가져옴
#     targets = examples['en']  # 'en' 열에서 영어 번역 텍스트를 가져옴
#     model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

#     # 타겟 텍스트도 토크나이징
#     labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

In [ ]:
# data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# # 6. 각 파트를 5 에폭씩 학습
# for i, part in enumerate(parts):
#     print(f"Training on part {i+1}...")

#     # 파트마다 전처리 및 토큰화
#     tokenized_part = part.map(preprocess_function, batched=True)

#     # Trainer 설정
#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=tokenized_part,  # 현재 파트 데이터셋으로 학습
#         eval_dataset=tokenized_part,   # 평가도 동일한 데이터셋으로 설정
#         data_collator=data_collator,
#         tokenizer=tokenizer
#     )

#     # api키는 이걸로 넣으면 됨 513a1f0c050fa7f60a76b5232e904d8df397082e

#     # 학습 시작
#     print(f"Training on dataset part {i+1}...")
#     try:
#         trainer.train()
#     except KeyboardInterrupt:
#         print(f"Training interrupted during part {i+1}. Last checkpoint saved.")

#     # 체크포인트 저장
#     trainer.save_model(f"/content/drive/MyDrive/translator/model_part_{i+1}")
#     print(f"Model checkpoint saved after part {i+1}")

In [ ]:
# for i, file_path in enumerate(file_paths):
#     # 각 파트의 데이터 불러오기
#     print(file_path)
#     print(f"Loading dataset part {i+1} from {file_path}")
#     #loaded_data = load_preprocessed_data(file_path)
#     loaded_data = load_preprocessed_data(file_path)

#     dataset = Dataset.from_dict({
#         'input_ids': loaded_data[0]['input_ids'],
#         'attention_mask': loaded_data[0]['attention_mask'],
#         'labels': loaded_data[0]['labels']
#     })
#     # train_dataset = loaded_dataset['train']
#     # eval_dataset = loaded_dataset['test']
#     # split_data = dataset.train_test_split(test_size=0.2)

#     # 6. 데이터 콜레이터 설정 (Seq2Seq에 맞는 형태)
#     data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

#     #split_data = loaded_data.train_test_split(test_size=0.2)

#     tokenized_part = part.map(preprocess_function, batched=True)


#     # 7. Trainer 설정
#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         #train_dataset=train_dataset,  # 학습 데이터셋
#         #eval_dataset=eval_dataset, # 평가 데이터셋 추가
#         train_dataset=tokenized_part,  # 현재 파트 데이터셋으로 학습
#         eval_dataset=tokenized_part,
#         data_collator=data_collator,
#         tokenizer=tokenizer
#     )

#     # 8. 학습 시작
#     print(f"Training on dataset part {i+1}...")
#     try:
#         trainer.train()
#     except KeyboardInterrupt:
#         print(f"Training interrupted during part {i+1}. Last checkpoint saved.")

#     # api키는 이걸로 넣으면 됨 513a1f0c050fa7f60a76b5232e904d8df397082e

#     # 체크포인트 저장
#     trainer.save_model(f"/content/drive/MyDrive/translator/model_part_{i+1}")
#     print(f"Model checkpoint saved after part {i+1}")


In [ ]:
model.save_pretrained("/content/drive/MyDrive/translator/final_model")
tokenizer.save_pretrained("/content/drive/MyDrive/translator/final_model")
print("Final model and tokenizer saved.")

#입력 및 예측


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 1. 토크나이저와 모델 로드 (학습이 된다면 변경 필요)
translation_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-tc-big-en-ko")
translation_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-tc-big-en-ko")

input_text="what should i say to you?"

inputs = translation_tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True, padding=True)
translated = translation_model.generate(**inputs)
translated_text = translation_tokenizer.decode(translated[0], skip_special_tokens=True)

# 6. 번역 결과 출력
print(f"번역된 영어 문장: {translated_text}")
